# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Session 9: LLM-based Apps with LangChain</font>

# <font color="#003660">Structured Outputs and Chains</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>

<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... will know what tool calling is. <br>
        ... will know how implement tools in LangChain and call tool chains.
    </font>
</div>
</p>

The following content is heavily inspired by the following excellent sources:

* [LangChain Academy](https://academy.langchain.com/)
* [LangChain Docs (Python)](https://python.langchain.com/)

In [ ]:
!pip install -U langchain langchain-community langchain-ollama ollama colab-xterm

In [ ]:
%load_ext colabxterm

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
%xterm # Copy this command in the Xterm starting below: HOST=0.0.0.0 ollama serve

In [ ]:
!ollama pull qwen2.5:1.5b

## Answering Maths Questions using LLMs

Using computers we usually want calculations to be made of precision. Unfortunately, LLMs have problems with solving mathematical reasoning tasks as you can see in the example below.

In [ ]:
import os
import re
from tqdm.notebook import tqdm
from typing import Optional

import torch
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

from langchain_core.tools import tool
from IPython.display import display, Latex, Markdown

DEVICE = "cuda:0" if torch.cuda.is_available() else "mps:0" if torch.mps.is_available() else "cpu"

In [ ]:
LLM_NAME = "qwen2.5:1.5b"

llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    seed=42
)

In [ ]:
query = "Also, what is 123 * 321?"
print(llm.invoke([{"role": "user", "content": query}]).content)

Wow, that's wrong. The right answer is 123 * 321 = 39483.

What can we do abour this? Usually, a human would use a calculator. Can LLMs also do that?

This is the concept of Tool-LLMs ([Qin et al. (2023)](https://doi.org/10.48550/arXiv.2307.16789)) or [function calling](https://www.promptingguide.ai/applications/function_calling).

*Function calling or tool usage* such as ToolAlpaca [Tang et al. (2023)](https://doi.org/10.48550/arXiv.2306.05301) allows to define functions or tools that the llm can use during execution.

First we have to define a `tool`, like the `add` and `multiply` functions below.

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

Now we need to bind these tools to the llm using LangChains `.bind_tools`.

In [ ]:
tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

Let's test it.

In [ ]:
query = "Also, what is 123 * 321?" # correct answer: 39483

print(llm_with_tools.invoke(query).model_dump_json(indent=4))

As we can see in the output, the LLM extracts the `"tool_calls"` with

```
{
    "name": "multiply",
    "args": {
        "a": 123,
        "b": 321
    },
    "id": "f98fe5d8-dd6a-4a67-938a-bc020fc27d15",
    "type": "tool_call"
}
```

a is 123 and b is 321, that is right.
But how can we really use this tool and how can we get the tool answer back.

First we need to store the answer of out ToolLLM. This can be done using the code below.

In [ ]:
query = "Also, what is 123 * 321?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

messages.append(ai_msg)

print(ai_msg.tool_calls)

Then we can iterate over the list of tool calls and run them while appending the results to the messages as in the code below.

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_call)
    print(tool_msg.model_dump_json(indent=4))

If we use all tool answers in the `messages`we can then prompt the LLM and get our final answer.

In [ ]:
print(llm_with_tools.invoke(messages).content)

# Your Task

In [ ]:
LLM_NAME = "qwen2.5:1.5b"

llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    seed=0
)

In [ ]:
query = "What is 222053/899?"
display(Markdown(llm.invoke([{"role": "user", "content": query}]).content.replace("\\[", "$").replace("\\]", "$").replace("\\(", "$").replace("\\)", "$")))

This answer seems to be wrong. The answer should be 247.

Can you extend the code to get a correct answer?

**Hint:** You will have to add a new tool to the code below.

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

# add a new tool here


In [ ]:
# maybe you have to change something here
tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
print(llm_with_tools.invoke(query).model_dump_json(indent=4))

In [ ]:
messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

messages.append(ai_msg)

print(ai_msg.tool_calls)

In [ ]:
# and some changes here?
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_call)
    print(tool_msg.model_dump_json(indent=4))

In [ ]:
print(llm_with_tools.invoke(messages).content)